In [1]:
# !conda install --yes numpy

In [11]:
import torch

In [12]:
torch.cuda.is_available()

True

In [13]:
!nvidia-smi

Sun Dec 18 19:00:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   49C    P0    N/A /  N/A |    751MiB /  3020MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [14]:
# %%capture
# !pip install datasets
# !pip install transformers
# !pip install torchaudio
# !pip install librosa
# !pip install jiwer

In [15]:
from datasets import load_dataset, load_metric, Audio

In [7]:
common_voice_train = load_dataset("common_voice", "ru", split="train+validation")
common_voice_test = load_dataset("common_voice", "ru", split="test")

Found cached dataset common_voice (/home/toxa/.cache/huggingface/datasets/common_voice/ru/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e)
Found cached dataset common_voice (/home/toxa/.cache/huggingface/datasets/common_voice/ru/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e)


In [8]:
#common_voice_test = common_voice_test.select([i for i in range(3000)]) #иначе оверфлоу по видеопамяти

In [9]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [10]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [11]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [12]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

Loading cached processed dataset at /home/toxa/.cache/huggingface/datasets/common_voice/ru/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-3cb53764b8952f07.arrow
Loading cached processed dataset at /home/toxa/.cache/huggingface/datasets/common_voice/ru/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-37e7f410261ff998.arrow


In [13]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence
0,втретьих существует финансовая спекуляция и спекуляция на финансовых рынках
1,тем не менее мое детство проведенное на юге судана было счастливым
2,малайзия признает важность развития человеческого потенциала в качестве стимула роста и развития
3,сахельские страны делают успехи в наращивании своего контртеррористического потенциала
4,необходимость юридически запретить производство расщепляющегося материала для оружейных целей получила широкое признание
5,для решения этих проблем крайне нужны слаженные и конкретные действия совета безопасности
6,это означает что после начала заседания в зале нельзя распространять никакие агитационные материалы
7,нами определены новые параметры для ответственности по защите
8,по своей сути мандат носит политический а не технический характер
9,две фундаментальные величины одной и той же размерности отличаются на семнадцать порядков


In [14]:
#def extract_all_chars(batch):
#    all_text = " ".join(batch["sentence"])
#    vocab = list(set(all_text))
#    return {"vocab": [vocab], "all_text": [all_text]}

In [15]:
#vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
#vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

In [16]:
#vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [17]:
#vocab_dict = {v: k for k, v in enumerate(vocab_list)}
#vocab_dict

In [18]:
#vocab_dict["|"] = vocab_dict[" "]
#del vocab_dict[" "]

In [19]:
#vocab_dict["[UNK]"] = len(vocab_dict)
#vocab_dict["[PAD]"] = len(vocab_dict)
#len(vocab_dict)

In [20]:
#import json
#with open('vocab.json', 'w') as vocab_file:
#    json.dump(vocab_dict, vocab_file)

In [4]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [5]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [6]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [24]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [25]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

In [26]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [27]:
import torch

In [28]:
torch.__version__

'1.12.1'

In [29]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, num_proc=1)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, num_proc=1)

Loading cached processed dataset at /home/toxa/.cache/huggingface/datasets/common_voice/ru/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-676a54156473a2e4.arrow
Loading cached processed dataset at /home/toxa/.cache/huggingface/datasets/common_voice/ru/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e/cache-d225c16ad59755b9.arrow


In [30]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [31]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [16]:
wer_metric = load_metric("wer")

/tmp/ipykernel_1397600/24688115.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [33]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [34]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "./model/.", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

In [35]:
model.freeze_feature_extractor()

/home/toxa/anaconda3/lib/python3.9/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1635: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [36]:
model.gradient_checkpointing_enable()

batch_size=20- загрузка 10гб первая карта

## дообучим на половине test, проверим на другой

In [37]:
common_voice_train = common_voice_test.select([i for i in range(len(common_voice_test)//2)])
common_voice_test = common_voice_test.select([i for i in range(len(common_voice_test)//2, len(common_voice_test))])

In [38]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-turkish-demo",
  output_dir="./wav2vec2-basee-ru-demo-3",
  group_by_length=True,
  per_device_train_batch_size=2,
  per_device_eval_batch_size=2,
  gradient_accumulation_steps=16,
  evaluation_strategy="steps",
  num_train_epochs=5,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-5, #сделаем поменьше т.к. батч тоже меньше
  warmup_steps=0,
  save_total_limit=2,
)

In [39]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

Using cuda_amp half precision backend


In [40]:
import numpy as np

In [41]:
trainer.train() 

/home/toxa/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4003
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 16
  Total optimization steps = 625
  Number of trainable parameters = 90221249
/home/toxa/anaconda3/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
100,0.493200,0.397872,0.492027
200,0.445000,0.400461,0.489944
300,0.437300,0.395986,0.487555
400,0.463700,0.389671,0.485194
500,0.527900,0.383696,0.484360
600,0.409000,0.379521,0.482416


***** Running Evaluation *****
  Num examples = 4004
  Batch size = 2
Saving model checkpoint to ./wav2vec2-basee-ru-demo-3/checkpoint-100
Configuration saved in ./wav2vec2-basee-ru-demo-3/checkpoint-100/config.json
Model weights saved in ./wav2vec2-basee-ru-demo-3/checkpoint-100/pytorch_model.bin
Feature extractor saved in ./wav2vec2-basee-ru-demo-3/checkpoint-100/preprocessor_config.json
/home/toxa/anaconda3/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
***** Running Evaluation *****
  Num examples = 4004
  Batch size = 2
Saving model checkpoint to ./wav2vec2-basee-ru-demo-3/checkpoint-200
Configuration saved in ./wav2vec2-basee-ru-demo-3/checkpoint-200/config.json
Model weights sa

TrainOutput(global_step=625, training_loss=0.47096892738342283, metrics={'train_runtime': 13375.749, 'train_samples_per_second': 1.496, 'train_steps_per_second': 0.047, 'total_flos': 1.1322251262865567e+18, 'train_loss': 0.47096892738342283, 'epoch': 5.0})

# тестим на другом датасете

In [32]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "model", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

In [17]:
data_test = load_dataset("bond005/sberdevices_golos_10h_crowd", split="test")

Using custom data configuration sberdevices_golos_10h_crowd-53c8b8379b590a4b
Found cached dataset parquet (/home/toxa/.cache/huggingface/datasets/bond005___parquet/sberdevices_golos_10h_crowd-53c8b8379b590a4b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [43]:
data_test[0]

{'audio': {'path': None,
  'array': array([ 0.        ,  0.        ,  0.        , ...,  0.0012207 ,
          0.00057983, -0.00082397]),
  'sampling_rate': 16000},
 'transcription': 'шестьдесят тысяч тенге сколько будет стоить'}

In [18]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\|]'
import torchaudio
resampler = torchaudio.transforms.Resample(16_000, 16_000)

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
    if batch["transcription"] is None:
        batch["transcription"] = ' '
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower()
    speech_array = batch['audio']['array']
    batch["speech"] = resampler(speech_array).squeeze()
    return batch
data_test = data_test.map(speech_file_to_array_fn)

Loading cached processed dataset at /home/toxa/.cache/huggingface/datasets/bond005___parquet/sberdevices_golos_10h_crowd-53c8b8379b590a4b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-28e9372e5ef0a74b.arrow


In [33]:
model.to('cuda')

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (2): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (3): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (4): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)

In [34]:
def evaluate(batch):
    inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_strings"] = processor.batch_decode(pred_ids)
    return batch

In [36]:
result = data_test.map(evaluate, batched=True, batch_size=1)

In [30]:
result[0]["pred_strings"]

'шестьдесят тысяч тенги сколько будет стоить'

In [31]:
print("WER: {:2f}".format(100 * wer_metric.compute(predictions=[[i] for i in result["pred_strings"]], references=[[i] for i in result["sentence"]])))

WER: 73.718126


In [19]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "wav2vec2-basee-ru-demo-3/checkpoint-600", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

In [20]:
model.to('cuda')

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (2): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (3): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (4): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)

In [24]:
result = data_test.map(evaluate, batched=True, batch_size=1)

  0%|          | 0/9994 [00:00<?, ?ba/s]

In [25]:
result[0]["pred_strings"]

'шестьдесят тысяч тенги сколько будет стоить'

In [26]:
print("WER: {:2f}".format(100 * wer_metric.compute(predictions=[[i] for i in result["pred_strings"]], references=[[i] for i in result["sentence"]])))

WER: 69.911050
